<a href="https://colab.research.google.com/github/VCHULANI15/pricing-options-with-machine-learning/blob/main/Pricing_Options_With_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing necessary libraries for manipulating data
import numpy as np
import pandas as pd
import seaborn as sns

#Plot Library
import matplotlib.pyplot as plt
import plotly.express as px
from plotnine import *

#Sklearn Library
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, median_absolute_error, mean_squared_error, r2_score, mean_absolute_error, classification_report
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

#Torch Data
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import norm
from itertools import product


In [5]:
np.random.seed(42)

n_samples = 100000

Spot=np.random.uniform(50,150,n_samples)
Strike=np.random.uniform(50,150,n_samples)
TTM=np.random.uniform(0.25,2,n_samples)

r=np.random.uniform(0.01,0.05,n_samples)

sigma = np.random.uniform(0.1, 0.4, n_samples) + (Strike < Spot) * np.random.uniform(0.05, 0.15, n_samples)

options_df = pd.DataFrame({
    'Spot Price': Spot,
    'Strike Price': Strike,
    'TTM': TTM,
    'Riks-Free Rate': r,
    'Volatility': sigma
})

In [12]:
def black_scholes_put_price(Spot, Strike, TTM, r, sigma, q =0):
  d1 = (np.log(Spot/Strike) + ( r- q + 0.5 * sigma ** 2) * TTM) / (sigma * np.sqrt(TTM))
  d2 = d1 - sigma * np.sqrt(TTM)

  put_price = Strike * np.exp(-r * TTM) * norm.cdf(-d2) - Spot * np.exp(-q * TTM) * norm.cdf(-d1)
  return put_price

options_df['BS_Put_Price'] = black_scholes_put_price(
    options_df['Spot Price'],
    options_df['Strike Price'],
    options_df['TTM'],
    options_df['Riks-Free Rate'],
    options_df['Volatility']
)

options_df.head()

,Spot Price,Strike Price,TTM,Riks-Free Rate,Volatility,BS_Put_Price
0,87.454012,108.077904,0.744529,0.016282,0.366686,24.191026
1,145.071431,102.697165,1.052684,0.013820,0.295664,2.102809
2,123.199394,85.103695,0.423627,0.015518,0.229967,0.029113
3,109.865848,99.321266,1.031965,0.028940,0.525141,15.590019
4,65.601864,86.509664,0.605392,0.045381,0.230878,19.065520
